In [32]:
import os
from secret_keys import openai_api_key, azure_openai_api_key, azure_openai_endpoint, azure_openai_version

In [33]:
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['AZURE_OPENAI_API_KEY'] = azure_openai_api_key
os.environ['AZURE_OPENAI_ENDPOINT'] = azure_openai_endpoint
os.environ['OPENAI_API_VERSION'] = azure_openai_version

In [34]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

In [43]:
#llm = AzureOpenAI(temperature=0.6, deployment_name='gpt-35-turbo')
#name = llm("I want to open a restaurent for Indian food. Suggest three fancy name for this")
#print("LLM:", name)

llm_chat = AzureChatOpenAI(temperature=0.6, deployment_name='gpt-4o-mini')
name = llm_chat.predict("I want to open a restaurent for Italian food. Suggest three fancy name for this")
print("Chat Model: ", name)

Chat Model:  Here are three fancy name suggestions for your Italian restaurant:

1. **Trattoria Bella Notte**  
   (Translation: "Beautiful Night") - Evokes a romantic and inviting atmosphere.

2. **Ristorante La Dolce Vita**  
   (Translation: "The Sweet Life") - Captures the essence of enjoying life through exquisite food.

3. **Osteria di Sogni**  
   (Translation: "Tavern of Dreams") - Suggests a place where culinary dreams come to life.

Let me know if you need more options!


In [59]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cusine'],
    template = "I want to open a restaurent for {cusine} food. Suggest a fancy name for it. Only one name please."
)

prompt_template_name.format(cusine="Mexican")

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggent some menu items for {restaurant_name}, Return it as comma separated list "
)

In [60]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm = llm_chat, prompt=prompt_template_name)
food_items_chain = LLMChain(llm=llm_chat, prompt=prompt_template_items)

In [61]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Mexican")
print(response)

Tacos al Pastor, Enchiladas Verdes, Chiles Rellenos, Mole Poblano, Guacamole with Chips, Ceviche de Camarón, Arroz con Pollo, Quesadillas de Huitlacoche, Tamales de Elote, Flan de Coco, Tres Leches Cake, Horchata, Agua Fresca de Jamaica, Sopes de Carnitas, Pozole Rojo.


In [62]:
name_chain = LLMChain(llm = llm_chat, prompt=prompt_template_name, output_key='restaurant_name')
food_items_chain = LLMChain(llm=llm_chat, prompt=prompt_template_items, output_key='menu_items')

In [64]:
from langchain.chains import SequentialChain

seq_chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cusine'],
    output_variables = ['restaurant_name', 'menu_items']
)

seq_chain(
    {'cusine' : 'Arabic'}
)

/tmp/ipykernel_130441/3879089836.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  seq_chain(


{'cusine': 'Arabic',
 'restaurant_name': '"Arabesque Delights"',
 'menu_items': 'Sure! Here are some menu items for "Arabesque Delights": \n\nHummus with Pita, Falafel Wrap, Shawarma Plate, Stuffed Grape Leaves, Tabouleh Salad, Lamb Kofta Skewers, Chicken Mandi, Grilled Eggplant with Tahini, Spiced Rice Pilaf, Baklava with Honey Syrup, Mint Tea, Fattoush Salad, Moroccan Tagine, Zaatar Manakish, Date and Walnut Delight.'}